In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


Original KNN

In [17]:
dataPath = '../../preprocess_train_dataset\pre06_train.csv'
dataframe = pd.read_csv(dataPath)

# Separate features (X) and label (y)
train_X = dataframe.drop('Transported', axis=1)
train_y = dataframe['Transported']

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

# Create and train the KNN classifier
# random assign a number
k = 3  
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(train_X, train_y)

# Evaluate the model
accuracy = knn.score(test_X, test_y)
print("Accuracy:", accuracy)

test_dataPath = '../../preprocess_test_dataset/pre06_test.csv'
PassengerId_path='../../preprocess_test_dataset/splittedTestData_01.csv'
df_test = pd.read_csv(test_dataPath)

#get PassengerId for result
df_PassengerId = pd.read_csv(PassengerId_path)
PassengerId = df_PassengerId['PassengerId']

# Predict on the test set
preds = knn.predict(df_test)

output_path = '../../output_prediction/'

submission = pd.DataFrame({'PassengerId': PassengerId, 'Transported': preds})
submission.to_csv(output_path+'KNN_Original.csv', index=False, index_label=False)



Accuracy: 0.7538815411155837


KNN with more approaches

In [18]:
dataPath = '../../preprocess_train_dataset\correlation.csv'
dataframe = pd.read_csv(dataPath)

Feature Scaling

In [19]:
train_X = dataframe.drop('Transported', axis=1)
train_y = dataframe['Transported']

# Exclude the output column from feature scaling
columns_to_scale = [col for col in train_X.columns]

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the data
train_X[columns_to_scale] = scaler.fit_transform(train_X[columns_to_scale])



In [20]:
# Display the modified dataset after feature scaling
train_X

,CryoSleep,HomePlanet_Earth,HomePlanet_Europa,Cabin_deck_B,Cabin_deck_C,Cabin_side_P,RoomService,Spa,VRDeck,spending_Categ_Low,spending_Categ_Medium,spending_Categ_High
0,0.0,0.0,1.0,1.0,0.0,1.0,0.000000,0.000000,0.000000,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.007608,0.024500,0.001823,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.003001,0.299670,0.002030,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.148563,0.007997,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.021149,0.025214,0.000083,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.073322,0.003066,0.0,0.0,1.0
8689,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0
8690,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000045,0.000000,0.0,1.0,0.0
8691,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.015753,0.134049,0.0,0.0,1.0


Parameter Tuning

In [21]:

# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

# Define the parameter grid
param_grid = {
    'n_neighbors': [3,5,7,9,11,13,15],
    'metric': ['euclidean', 'manhattan','minkowski']
}

# Initialize the K-Nearest Neighbors classifier
knn = KNeighborsClassifier()

# Perform grid search to find the best parameters
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Make predictions on the test data using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the accuracy of the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy (Best Model): {:.2f}%".format(accuracy * 100))

#  Print the best parameters found by grid search
print("Best Parameters:", grid_search.best_params_)


Accuracy (Best Model): 76.88%
Best Parameters: {'metric': 'euclidean', 'n_neighbors': 15}


prediction

In [23]:



test_dataPath = '../../preprocess_test_dataset/pre06_test.csv'
df_test = pd.read_csv(test_dataPath)

PassengerId_path='../../preprocess_test_dataset/splittedTestData_01.csv'
#get PassengerId for result
df_PassengerId = pd.read_csv(PassengerId_path)
PassengerId = df_PassengerId['PassengerId']

print(df_test.shape)
print(df_test.columns)

(4277, 23)
Index(['CryoSleep', 'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'Cabin_deck_A', 'Cabin_deck_B', 'Cabin_deck_C', 'Cabin_deck_D',
       'Cabin_deck_E', 'Cabin_deck_F', 'Cabin_deck_G', 'Cabin_deck_T',
       'Cabin_side_P', 'Cabin_side_S', 'Age', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'spending_Categ_Low',
       'spending_Categ_Medium', 'spending_Categ_High'],
      dtype='object')


In [24]:

X_dataframe = pd.DataFrame(df_test)

dataframe.iloc[:, :-1]

# Select only the columns present in the training dataset
X_private_selected = X_dataframe[dataframe.iloc[:, :-1].columns]


# Make predictions on the test dataset
preds = grid_search.predict(X_private_selected)

# perpare output result
PassengerId = df_PassengerId['PassengerId']
output_path = '../../output_prediction/'

submission = pd.DataFrame({'PassengerId': PassengerId, 'Transported': preds})
submission.to_csv(output_path+'KNN.csv', index=False, index_label=False)
